In [ ]:
import jqdatasdk as jq
import akshare as ak
import pandas as pd
from datetime import datetime, timedelta
import pandas as pd
import copy
from datetime import date, datetime
from pyxirr import xirr

In [ ]:
jq.auth('15158172001', '3EKwG4L1ujo4')

In [ ]:
# normalize_code(['518880'])
his_price = jq.get_price(security='518880.XSHG', frequency='1m', start_date='2024-01-19', end_date='2025-01-23')
his_price.to_csv('data-20240119-2025-01-23.csv')

In [ ]:
# ak.stock_zh_a_hist(symbol="518880", period="daily", start_date="20240117", end_date='20250123', adjust="")
# ak.fund_open_fund_info_em(symbol='518880', indicator="单位净值走势")
# ak.fund_etf_fund_info_em(fund="518880", start_date="20240117", end_date="20250426")
df = ak.fund_etf_hist_em(symbol="518880", period="daily", start_date="20230101", end_date="20250426", adjust="")

n, k = 30, 2

df['标准差'] = df['收盘'].rolling(window=n).std(ddof=1).round(3)
df['MID'] = df['收盘'].rolling(window=n).mean().round(3)
df['UP'] = df['MID'] + k * df['标准差']
df['LOW'] = df['MID'] - k * df['标准差']

df.tail(10)

In [ ]:
def _eval(curr_time, portfolio):
    interval = portfolio['interval']
    max_pos = portfolio['max_pos']
    
    boll_row = df[df['日期'] == curr_time.strftime('%Y-%m-%d')]
    if len(boll_row) == 0:
        # 非交易日
        return portfolio
        
    try:
        boll = boll_row['MID'].item()
    except Exception as e:
        print(curr_time)
        raise e
    min_loss, max_loss = portfolio['max_loss']
    bp = round(boll / (1 - max_loss), 3)
    bp_low = round(boll / (1 - min_loss), 3)
    sl = boll
    
    op, cp, hp, lp, vol, money = his_price.loc[curr_time].values
    trade = {
        'date': curr_time, 
        'sl': sl,
        'bp': bp,
        'op': op,
        'cp': cp,
        'hp': hp,
        'lp': lp,
        'vol': vol
    }
    # 选择价格
    price = cp
    if price <= sl:
        # 触发止损
        if len(portfolio['position']) > 0:
            # print(f"{curr_time} sell\t{sl, bp}\t{op, cp, hp, lp}")
            for buy_trade in portfolio['position']:
                pos = buy_trade['pos']
                sell_trade = copy.deepcopy(trade)
                sell_trade.update({
                    'op': 'sell',
                    'pos': pos, 
                    'money': pos * price,
                    'price': price, 
                })
                portfolio['trades'].append({
                    'buy_trade': buy_trade,
                    'sell_trade': sell_trade
                })
            portfolio['position'] = []
            portfolio['last_sell'] = curr_time
    elif sl < price and price <= bp and bp_low < price:
        # 价格在布林带区间，且价格在 bp_low 之上（防止频繁扫单），触发买入
        if portfolio['cash'] > 0 and (curr_time - portfolio['last_buy']).days >= interval and len(portfolio['position']) < max_pos:
            # print(f"{curr_time} buy \t{sl, bp}\t{op, cp, hp, lp}")
            # 仓位是 100 手的整数倍
            pos = portfolio['cash'] // (price * 100) * 100
            portfolio['cash'] = 0
            trade.update({
                'op': 'buy',
                'pos': pos, 
                'money': pos * price,
                'price': price, 
            })
            portfolio['position'].append(trade)
            portfolio['last_buy'] = curr_time
            portfolio['press'] = max(portfolio['press'], len(portfolio['position']))
    elif bp < price:
        # 等待机会
        pass

    return portfolio
    
    

start_time = datetime(2024, 1, 19, 9, 31, 0)
end_time = datetime(2025, 1, 22, 15, 00, 00)

def regression(start_time, end_time, max_loss=(0.01, 0.02), interval=5, max_pos=1000):
    current_time = start_time
    cash_per_pos = 10000
    portfolio = {
        'position': [],
        'last_buy': datetime(2000, 1, 1, 9, 31, 0),
        'last_sell': datetime(2000, 1, 1, 9, 31, 0),
        'press': 0,
        'cash': cash_per_pos,
        'max_loss': max_loss,
        'interval': interval,
        'max_pos': max_pos,
        'trades': []
    }
    while current_time <= end_time:
        budge = _eval(current_time, portfolio)
        if current_time.hour == 11 and current_time.minute == 30:
            # 11:30 上午结束，跳转至下午 13:01
            current_time += timedelta(minutes=91)
        elif current_time.hour == 15 and current_time.minute == 00:
            # 15:00 交易结束，跳转至第二天 9:31
            current_time += timedelta(minutes=1111)
            # 预算恢复至 1000
            portfolio['cash'] = cash_per_pos
        else:
            current_time += timedelta(minutes=1)
    return portfolio


lr_list = [(0, 0.02)]       # [(0,0.01), (0, 0.02), (0, 0.05), (0, 0.1)]
interval_list = [0]         # [3, 5, 10, 15, 20]
max_pos_list = [1, 2, 3, 4, 5]

result_list = []
for lr in lr_list:
    for interval in interval_list:
        for max_pos in max_pos_list:
            ret = regression(start_time, end_time, lr, interval, max_pos)
            result_list.append(ret)

In [ ]:
def save(portfolio):
    op, cp, hp, lp, vol, money = his_price.loc[end_time].values
    trade = {
        'date': end_time, 
        'sl': None,
        'bp': None,
        'op': op,
        'cp': cp,
        'hp': hp,
        'lp': lp,
        'vol': vol
    }
    for buy_trade in portfolio['position']:
        pos = buy_trade['pos']
        sell_trade = copy.deepcopy(trade)
        sell_trade.update({
            'op': 'sell',
            'pos': pos, 
            'money': pos * cp,
            'price': cp, 
        })
        portfolio['trades'].append({
            'buy_trade': buy_trade,
            'sell_trade': sell_trade
        })
        
    trades = pd.json_normalize(portfolio['trades'])
    
    trades['profit'] = trades['sell_trade.money'] - trades['buy_trade.money']
    
    buy_dates = trades['buy_trade.date']
    buy_amounts = trades['buy_trade.money'] * -1

    sell_dates = trades['sell_trade.date']
    sell_amounts = trades['sell_trade.money']

    dates = pd.concat([buy_dates, sell_dates])
    amounts = pd.concat([buy_amounts, sell_amounts])
    
    rate = round(xirr(dates, amounts) * 100, 2)
    
    
    trades = trades[['buy_trade.date', 'buy_trade.sl','buy_trade.pos', 'buy_trade.price', 'sell_trade.date', 'sell_trade.sl', 'sell_trade.pos', 'sell_trade.price', 'profit']]


    max_loss = str(portfolio['max_loss'])
    interval = str(portfolio['interval'])
    
    # trades.to_csv(f'data-{max_loss}-{interval}.csv')

    return {
        'max_loss': portfolio['max_loss'],
        'interval': portfolio['interval'],
        'max_pos': portfolio['max_pos'],
        'press': portfolio['press'],
        'trades': len(trades),
        'profit': trades['profit'].sum().round(2),
        'xirr': rate
    }

report_list = []
for res in result_list:
    portfolio = copy.deepcopy(res)
    report_list.append(save(portfolio))

report = pd.json_normalize(report_list)

report

In [ ]:
mk = his_price.loc[datetime.strptime('2024-01-19 09:31:00', "%Y-%m-%d %H:%M:%S")]
mid = df[df['日期'] == '2024-01-18']['MID'].item()


In [ ]:
import pickle

# 保存对象到文件
def save_to_pickle(obj, filename):
    with open(filename, 'wb') as f:
        pickle.dump(obj, f)
def load_from_pickle(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)


In [ ]:

# save_to_pickle(result_list, 'result_list.pkl')
result_list = load_from_pickle('result_list.pkl')


In [ ]:
data = pd.DataFrame( {'buy_date': [date(2020, 1, 1), date(2022, 1, 1)], 'buy_amount': [100, 110], 'sell_date': [date(2021, 1, 1), date(2023, 1, 1)], 'sell_amount': [120, 100]})
dates = pd.concat([data['buy_date'], data['sell_date']])
amounts = pd.concat([data['buy_amount'] * -1, data['sell_amount']])

n = 5
profit = 4743.6

print(f"XIRR: {xirr([start_time, end_time], [n * -10000, n * 10000 + profit]):.2%}")